# Application Diagnosis Runbook

Use this runbook to diagnose a failed web server

Is the container running?

In [ ]:
!docker container list

Is the certificate expired?

In [ ]:
import requests
from datetime import datetime

def get_cert(url):
    try:
        with requests.get(url, stream=True) as response:
            certificate_info = response.raw.connection.sock.getpeercert()
            #print(certificate_info)

            subject = dict(x[0] for x in certificate_info['subject'])
            issuer = dict(x[0] for x in certificate_info['issuer'])
            sans = [x[1] for x in certificate_info['subjectAltName']]

            result = {}
            result['status'] = "success"
            result['commonName'] = subject['commonName']
            result['SANs'] = sans
            result['notBefore'] = datetime.strptime(certificate_info['notBefore'], '%b %d %H:%M:%S %Y GMT')
            result['notAfter'] = datetime.strptime(certificate_info['notAfter'], '%b %d %H:%M:%S %Y GMT')
            result['serialNumber'] = certificate_info['serialNumber']

            return result
    except Exception as e:
       result = {}
       result['status'] = "error"
       result['message'] = e.args
       return result

url = "https://robrich.org/"
#url = "https://www.azgivecamp.org/"
#url = "https://azgivecamp.org/"
cert = get_cert(url)
#print(cert)
if cert["status"] == "error":
    print(f'Error getting cert from {url}')
    print(cert["message"])
elif cert["notBefore"] < datetime.utcnow() and cert["notAfter"] > datetime.utcnow():
    print(url, "Cert is valid")
elif cert["notBefore"] >= datetime.utcnow():
    print(url, "Cert is not yet valid", cert["notBefore"])
elif cert["notAfter"] <= datetime.utcnow():
    print(url, "Cert is expired", cert['notAfter'])
else:
    print(f'Cert is invalid for {url}')

Can we connect to the database?

In [ ]:
import psycopg2

pg_conn = psycopg2.connect(host="localhost",user="postgres",password="postgres",dbname="pgdb")
cur = pg_conn.cursor()
cur.execute("select count(*) from movies")
rows = cur.fetchone()[0]
print(f'Rows in the movies table: {rows}')
pg_conn.close()

Can we get the home page?

In [ ]:
import requests

res = requests.get(url)
print(f'HTTP Status Code: {res.status_code}')
print(res.content)

In [ ]:
%%html
<iframe src="https://robrich.org/" width="1000" height="800"></iframe>